In [1]:
import pandas as pd
import pyomo
import pulp
from pulp import *
from pyutilib.services import register_executable, registered_executable
register_executable(name='glpsol')

In [2]:
# configs

data_path = '..\data\dados_trajeto.csv'

m = 12000
rho = 1.2754
Cd = 0.36
Cr = 0.9
Af = 6.48
g = 9.81
Cfc = 2
Cbt = 0.92

## 1.1 Função Objetivo

O = min$\sum_{i=1}^{N}$c<sub>fc</sub>*P<sub>fc</sub>(t<sub>i</sub>) + c<sub>b</sub>*P<sub>b</sub>(t<sub>i</sub>)

Em que: <br> 

P<sub>fc</sub> = Potência fornecida pela célula combustível <br>
c<sub>fc</sub> = custo de operação RS/potencia da célula combustível <br>
P<sub>b</sub> = Potência fornecida pela bateria <br>
c<sub>b</sub> = custo de operação RS/potencia da bateria <br>

## 1.2 Variáveis de decisão

- P<sub>fc</sub>(t<sub>i</sub>)
- P<sub>b</sub>(t<sub>i</sub>)

## 1.3 Parâmetros

* c<sub>b</sub>
* c<sub>fc</sub>
* t = tempo da Viagem
* P<sub>d</sub> = Potência Demandada

## 1.4 Restrições

P<sub>fc</sub>*(t) + P<sub>b</sub>*(t) = Ptr(t) <br>

## 2. Modelando a demanda de potência do veículo

**Força de arrasto (Fa)**

Fa(t) = 0,5.ρ.Cd.Af.(v(t))²

em que:
* ρ é a densidade do ar
* Cd é o coeficiente de arrasto para o ônibus
* Af é a área da superfície frontal do veículo
* v(t) é a velocidade do veículo em função do tempo


*(Para fins de facilitar os cálculos nesta primeira fase de implementação, será considerado uma inclinação de **0 graus** no trajeto)*

**Força de resistência ao rolamento (Fr)**

Fr(t) = (m+m<sub>b</sub>+m<sub>fc</sub>).g.Cr.cos(§(t))

em que:
* m é a massa do veículo
* m<sub>b</sub> é a massa da bateria
* m<sub>fc</sub> é a massa da célula combustível
* g é a aceleração da gravidade (9,81 m/s²)
* Cr é o coeficiente de atrito de rolamento
* §(t) é a inclinação do trajeto em função do tempo

**Força gravitacional (Fg)**

Fg(t) = (m+m<sub>b</sub>+m<sub>fc</sub>).g.Cr.sin(§(t))
(Será nula uma vez que foi considerado um angulo de inclinação de 0 graus)

**Força de aceleração (Facc)**

Facc(t) = (m+m<sub>b</sub>+m<sub>fc</sub>).(dvc(t)/dt)


A Força de tração é a soma de todas as forças citadas acima:

Ftr(t) = Facc(t) + Fg(t) + Fr(t) + Fa(t)

Assim, a demanda de potência pode ser dada por:

Ptr(t) = Ftr(t).v(t)

Usando os dados do TG do vinicius apenas como base para o teste

## 3. Otimização

### 3.1 Carregando os dados

In [3]:
df = pd.read_csv(data_path, sep = ';')

df['tempo_s'] = df['tempo_s'] - 10
df['distancia_m'] = df['distancia_m'] - 155.42

delta = df.diff()

df['velocidade'] = delta['distancia_m']/delta['tempo_s']

delta = df.diff()

df['acc'] = abs(delta['velocidade'])/delta['tempo_s']
df.fillna(0, inplace=True)

df['Fa'] = 0.5*rho*Cd*Af*((df['velocidade'])**2)
df['Fr'] = m*g*Cr
df['Facc'] = m*df['acc']
df['Ftr'] = df['Fa'] + df['Fr'] + df['Facc']
df['pot'] = (df['Ftr']*df['velocidade'])/1000

In [4]:
df

,tempo_s,distancia_m,velocidade,acc,Fa,Fr,Facc,Ftr,pot
0,0,0.00,0.00,0.000000e+00,0.000000,105948.0,0.000000e+00,105948.000000,0.000000
1,1,11.95,11.95,0.000000e+00,212.436792,105948.0,0.000000e+00,106160.436792,1268.617220
2,2,23.91,11.96,1.000000e-02,212.792483,105948.0,1.200000e+02,106280.792483,1271.118278
3,3,35.86,11.95,1.000000e-02,212.436792,105948.0,1.200000e+02,106280.436792,1270.051220
4,4,47.82,11.96,1.000000e-02,212.792483,105948.0,1.200000e+02,106280.792483,1271.118278
...,...,...,...,...,...,...,...,...,...
1094,1094,5946.14,11.87,1.000000e-02,209.601971,105948.0,1.200000e+02,106277.601971,1261.515135
1095,1095,5958.03,11.89,2.000000e-02,210.308891,105948.0,2.400000e+02,106398.308891,1265.075893
1096,1096,5969.92,11.89,9.094947e-13,210.308891,105948.0,1.091394e-08,106158.308891,1262.222293
1097,1097,5981.83,11.91,2.000000e-02,211.017001,105948.0,2.400000e+02,106399.017001,1267.212292


### 3.2 Construindo o problema

In [7]:
# Definindo o problema
problema = pulp.LpProblem('Dimensionamento_onibus', pulp.LpMinimize)

In [ ]:
# Criando as variáveis de decisão
des_vars = []

for rownum, row in df.iterrows():
            variablestr_1 = str('Pfc' + str(rownum)) 
            variablestr_2 = str('Pb' + str(rownum)) 
            
            variable_1 = pulp.LpVariable(str(variablestr_1), lowBound = 0, cat= 'Continuous') 
            variable_2 = pulp.LpVariable(str(variablestr_2), lowBound = 0, cat= 'Continuous') 
            
            des_vars.append(variable_1)
            des_vars.append(variable_2)
            
            
print ("Número total de variáveis de decisão: " + str(len(des_vars)))

In [ ]:
# Criando função objetivo
custo_total = ''

for i in range(0,len(des_vars), 2):
    
    custo_total += Cfc*des_vars[i] + Cbt*des_vars[i+1]

problema += custo_total

In [ ]:
# Restrições
pot_total = ''
demanda_total = 0

for i in range(0,len(des_vars), 2):

    pot_total += des_vars[i] + \
                 des_vars[i+1] - \
                 (2.5*des_vars[i] + 5.6/32.242*des_vars[i+1])*g*Cr - \
                 (2.5*des_vars[i] + 5.6/32.242*des_vars[i+1])*df['acc'][(i/2)]
    
    demanda_total += df['pot'][(i/2)] 

In [ ]:
resultado = problema.solve(pulp.GLPK(msg = 0))
assert resultado == pulp.LpStatusOptimal
print("Status:", pulp.LpStatus[problema.status])
print("solução ótima: ", pulp.value(problema.objective))